In [3]:
from scipy.stats import kstest
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import chardet
data = '../data/'

In [4]:
world = pd.read_csv(data + "world_country.csv")
baseCountry = world[['name','alpha-3']]
baseCountry = baseCountry.rename(columns={'alpha-3': 'iso'})
baseCountry

,name,iso
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM
...,...,...
244,Wallis and Futuna,WLF
245,Western Sahara,ESH
246,Yemen,YEM
247,Zambia,ZMB


In [5]:
incomes = pd.read_csv(data + "worldincome.csv", parse_dates=True)
#baseCountry = baseCountry.rename(columns={'country': 'iso'})
incomes.describe()
incomes

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297
...,...,...,...,...,...,...
11594,COD,2008,96,100,"810,6233","303,19305"
11595,COD,2008,97,100,"911,7834","303,19305"
11596,COD,2008,98,100,"1057,8074","303,19305"
11597,COD,2008,99,100,"1286,6029","303,19305"


In [6]:
incomes["year_survey"].unique() # timeline in df 

array([2008, 2009, 2010, 2007, 2006, 2011, 2004])

In [7]:
representCountry = len(incomes)
totalCountriesInWorld = 195
populationCoveradeByDf = (representCountry / totalCountriesInWorld) * 100
populationCoveradeByDf

5948.205128205128

On resence 195 pays dans le monde le dataset représente 160 pays nous avons 59% qui sont représenter dans le dataset

nous allons clear le dataset voir si il y a des valeurs manquantes etc ..

In [8]:
dfDate = incomes.copy()
dfDate.drop_duplicates(subset='year_survey', keep='first',inplace=True)
dfDate.sort_values(by='year_survey') # la frise chrnologique commence en 2004 et fini en 2011 il manque les donnée de 2005

,country,year_survey,quantile,nb_quantiles,income,gdpppp
9899,SYR,2004,1,100,"143,18066",4512
3600,GHA,2006,1,100,"63,157047","1380,1182"
900,BIH,2007,1,100,"808,3991",7472
0,ALB,2008,1,100,"728,89795",7297
600,BFA,2009,1,100,"138,40517","1085,5139"
700,BGD,2010,1,100,"276,93503",1233
3900,GTM,2011,1,100,"38,463615",4367


In [ ]:
incomes.info()

In [9]:
incomes.replace(',','.', regex=True, inplace=True)
incomes['income'] = incomes['income'].astype(float)
incomes['gdpppp'] = incomes['gdpppp'].astype(float)



In [23]:
incomes[incomes['country']=='ALB'].sort_values(by='income') # j'ai pris le premiers pays du dataset pour connaitre le nombre de quantile 
## nb quantiles toujours = 100

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.0
1,ALB,2008,2,100,916.66235,7297.0
2,ALB,2008,3,100,1010.91600,7297.0
3,ALB,2008,4,100,1086.90780,7297.0
4,ALB,2008,5,100,1132.69970,7297.0
...,...,...,...,...,...,...
95,ALB,2008,96,100,6231.09960,7297.0
96,ALB,2008,97,100,6822.31840,7297.0
97,ALB,2008,98,100,7775.89100,7297.0
98,ALB,2008,99,100,9351.75100,7297.0


In [11]:
numberOfCountry = incomes.groupby('country')
numberOfCountry.count() # nb pays 

,year_survey,quantile,nb_quantiles,income,gdpppp
country,,,,,
ALB,100,100,100,100,100
ARG,100,100,100,100,100
ARM,100,100,100,100,100
AUT,100,100,100,100,100
AZE,100,100,100,100,100
...,...,...,...,...,...
VEN,100,100,100,100,100
VNM,100,100,100,100,100
XKX,100,100,100,100,0


In [12]:
incomes.count() ## si il y a 116 il manque une ligne

country         11599
year_survey     11599
quantile        11599
nb_quantiles    11599
income          11599
gdpppp          11399
dtype: int64

In [13]:
countQuantile = incomes.pivot_table(index='country').sort_values(by='quantile').reset_index()[['country', 'quantile']]
diffQuantile = countQuantile[countQuantile['quantile'] != 50.50000]
diffQuantile ## quantile manquant

,country,quantile
115,LTU,50.59596


In [14]:
ltu = incomes[incomes['country']=='LTU']
## voir la ligne manquante 
##https://www.appsloveworld.com/numpy/100/21/check-the-missing-integers-from-a-range-in-python
missedQuantile = np.setdiff1d(list(range(1, 101)), list(ltu['quantile']))

missedQuantile

array([41])

In [15]:
if len(missedQuantile) != 0: 
    ## on fait une moyenne de ces deux valeurs 
    getRowsLtu = ltu[ltu['quantile'].isin([missedQuantile[0]-1,missedQuantile[0]+1])]
    getRowsLtu
    ## creation de la row 
    meanRowLtu = getRowsLtu.pivot_table(index='country').reset_index()
    ## insérer la row manquante
    incomes = pd.concat([incomes, meanRowLtu]).reset_index().drop(columns='index')

   

In [16]:
## valeur manquantes gdppppp
showMissGdp = incomes[incomes['gdpppp'].isnull()]
showMissGdp

percentDataMissing = round(len(showMissGdp) / len(incomes) * 100, 1) 
percentDataMissing # les valeurs manquantes gdpppp represente 1.7 % du dataset on décide de les supprimer

showMissGdp


,country,year_survey,quantile,nb_quantiles,income,gdpppp
5800,XKX,2008,1,100,437.89370,NaN
5801,XKX,2008,2,100,508.17133,NaN
5802,XKX,2008,3,100,591.82820,NaN
5803,XKX,2008,4,100,668.00000,NaN
5804,XKX,2008,5,100,730.40220,NaN
...,...,...,...,...,...,...
11294,PSE,2009,96,100,2763.88480,NaN
11295,PSE,2009,97,100,3077.83330,NaN
11296,PSE,2009,98,100,3449.22240,NaN
11297,PSE,2009,99,100,4165.99700,NaN


In [17]:
if showMissGdp['gdpppp'].isnull().sum() > 0:
    incomes.drop(showMissGdp.index, inplace=True)
    

In [18]:
## on enregistre le dataset clear pour la suite de nos exercices

In [19]:
incomes.reset_index()
incomes.to_csv(data+ 'worldincome_clear.csv', index=False)
incomes


,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,728.89795,7297.00000
1,ALB,2008,2,100,916.66235,7297.00000
2,ALB,2008,3,100,1010.91600,7297.00000
3,ALB,2008,4,100,1086.90780,7297.00000
4,ALB,2008,5,100,1132.69970,7297.00000
...,...,...,...,...,...,...
11595,COD,2008,97,100,911.78340,303.19305
11596,COD,2008,98,100,1057.80740,303.19305
11597,COD,2008,99,100,1286.60290,303.19305
11598,COD,2008,100,100,2243.12260,303.19305


#  DATA GINI

In [22]:
giniFile = data + "indicegini.csv";
#gini = pd.read_csv(, parse_dates=True, encoding = 'latin1') 
#gini
gini = pd.read_csv(giniFile)

gini


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 10: invalid start byte

In [51]:
giniCp = gini.copy()
giniCp.head()
## les données du world income sont de 2004 to 2011
clearGini = giniCp[['Country Name', 'Country Code','2004','2005','2006','2007','2008','2009','2010','2011']]
clearGini


,Country Name,Country Code,2004,2005,2006,2007,2008,2009,2010,2011
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,42.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,NaN,31.2,30.3,NaN,NaN,31.8,33.3,27.8
262,"Yemen, Rep.",YEM,NaN,34.7,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,NaN,64.8,NaN,NaN,63.0,NaN,63.4,NaN
264,Zambia,ZMB,54.3,NaN,54.6,NaN,NaN,NaN,55.6,NaN


In [57]:
transformGini = clearGini.melt(id_vars=['Country Name', 'Country Code'], var_name='date', value_name='valeur')
groupGini = transformGini.groupby(['Country Name', 'Country Code', 'date']).sum().reset_index()
groupGini
#groupGini.to_csv(data + 'newGini.csv', index=False)


,Country Name,Country Code,date,valeur
0,Afghanistan,AFG,2004,0.0
1,Afghanistan,AFG,2005,0.0
2,Afghanistan,AFG,2006,0.0
3,Afghanistan,AFG,2007,0.0
4,Afghanistan,AFG,2008,0.0
...,...,...,...,...
2123,Zimbabwe,ZWE,2007,0.0
2124,Zimbabwe,ZWE,2008,0.0
2125,Zimbabwe,ZWE,2009,0.0
2126,Zimbabwe,ZWE,2010,0.0


In [58]:
groupGini = groupGini.rename(columns={'valeur': 'gini_value'})
groupGini = groupGini.rename(columns={'Country Name': 'country_name'})
groupGini = groupGini.rename(columns={'Country Code': 'country_code'})
groupGini

,country_name,country_code,date,gini_value
0,Afghanistan,AFG,2004,0.0
1,Afghanistan,AFG,2005,0.0
2,Afghanistan,AFG,2006,0.0
3,Afghanistan,AFG,2007,0.0
4,Afghanistan,AFG,2008,0.0
...,...,...,...,...
2123,Zimbabwe,ZWE,2007,0.0
2124,Zimbabwe,ZWE,2008,0.0
2125,Zimbabwe,ZWE,2009,0.0
2126,Zimbabwe,ZWE,2010,0.0


In [62]:
groupGini = groupGini[groupGini['gini_value'] != 0.0]
groupGini.reset_index
groupGini

,country_name,country_code,date,gini_value
25,Albania,ALB,2005,30.6
28,Albania,ALB,2008,30.0
39,Algeria,DZA,2011,27.6
60,Angola,AGO,2008,42.7
80,Argentina,ARG,2004,48.4
...,...,...,...,...
2105,"Yemen, Rep.",YEM,2005,34.7
2112,Zambia,ZMB,2004,54.3
2114,Zambia,ZMB,2006,54.6
2118,Zambia,ZMB,2010,55.6


In [ ]:
income = pd.merge(baseCountry, incomes, on='iso')
# Vérifier les valeurs manquantes dans le second dataset
missing_values = merged_df['iso'].isnull().sum()
missing_values